# Genomic-ULMFiT Methods 0: Data Representation, Model Architecture, Regularization and Training

Karl Heyer

## Introduction

Genomic-ULMFiT (G-ULMFiT) is a method for training deep genomic sequence classification models that shows competitive or improved performance over previous published results. This technique allows us to solve problems like:
 * Does this genomic sequence contain a promoter?
 * Is this RNA sequence coding RNA or non-coding RNA?
 * What genus does this sequencing read belong to?

This method is based on ULMFiT [1] - a transfer learning method for NLP tasks. Transfer learning is the process of taking a model trained to solve one task as the basis for solving another task. This means that rather than training a model from scratch for the second task, we initialize the model with weights learned from the initial task, then *fine tune* the weights towards the second task. Transfer learning has been extensively applied in the field of computer vision. For example, one might train a classification model on ImageNet data, then fine tune that model for classification of satellite imagery. 

Transfer learning in NLP domains has historically been restricted to embeddings. NLP models would have embedding layers initialized with pretrained weights from word2vec, GloVe, or some other source. ULMFiT extends this to transferring learned weights for multiple layers, to great effect. Importantly, the initial model is trained in an unsupervised fashion before being fine tuned for supervised classification on a labeled dataset. This means that our model performance is not restricted by the availability of labeled data. From a genomics perspective, this allows us to train the initial model on large amounts of unlabeled sequence data, then transfer the learned weights to a classification model that is fine tuned on a smaller dataset. This allows G-ULMFiT to leverage the huge amount of unlabeled sequence data available to produce accurate results on small labeled datasets using __only genomic sequences as input__. The initial model is also general and reusable. It can be fine tuned towards any number of classification tasks and datasets.

This document covers the theory behind G-ULMFiT and practical considerations for structuring data and training models. This document is written with the following goals in mind:
   * Cover the theory of the ULMFiT process and considerations taken applying it to genomic data
   * Explain the model architectures used, the theory behind them, and important hyperparameters
   * Describe practical methods for training models quickly and achieving high performance, including regularization and learning rate scheduling
    
This document is not intended to be a code walkthrough. Relevant code is shown in the notebooks in the [E. coli](https://github.com/kheyer/Genomic-ULMFiT/tree/master/Bacteria/E.%20Coli) directory.

This document is structured in the following sections:
   * Genomic Sequence Data Representation details preprocessing steps of preparing genomic data before sending it as input to a model
   * ULMFiT Overview describes the overall method and the template for what we want to achieve
   * Model Architecture covers Genomic Language Models, Genomic Classification Models, the layers that comprise them, and important hyperparameter choices in model design
   * Regularization details the many types of regularization at play in the model and how to tune them effectively
   * Training covers the ULMFiT process in detail, as well as learning rate schedules and training phases


## Table of Contents
1. Genomic Sequence Data Representation
    * 1.1 Genomic Tokenization
    * 1.2 Genomic Numericalization
2. ULMFiT Overview

## 1. Genomic Sequence Data Representation

If we want to train a sequence model on genomic data, the first thing we need to figure out is how to process the data into a form that can be used by a neural network. We need to turn sequence data into a numerical form that can be manipulated mathematically. We do this in two steps - __tokenization__ and __numericalization__.

Tokenization is the process of breaking the sequence down into sub-units or tokens. Numericalization is the process of mapping tokens to integer values.

### 1.1 Genomic Tokenization

How do we break genomic data into tokens? A common way is to tokenize by nucleotide [2,3,4,5,6,7]. Single nucleotide tokenization would process the sequence `ATCGCGTACG` into `A T C G C G T A C G`. This works, but it gives a very restricted representation of genomic sub-units. This looks at every nucleotide in a vacuum. It essentially says every `A` should be treated the same, regardless of where it appears or in what context it appears.

A representation that allows for more nuance is to tokenize by k-mers instead. This approach has been used by [8,9,10]. We could tokenize the sequence `ATCGCGTACGATCCG` into:
 * 3-mers: `ATC GCG TAC GAT CCG`
 * 4-mers: `ATCG CGTA CGAT`
 * 5-mers: `ATCGC GTACG ATCCG`
 
Or some other k-mer size. Notice in the above that the sequence is truncated to the last whole k-mer. 

Another parameter in tokenization is the stride between k-mers. Stride is defined as the frame shift between k-mers relative to the sequence being tokenized. In the above example, there was no overlap between k-mers, but this does not have to be the case. Consider tokenizing the sequence `ATCGCGTACGATCCG` with a k-mer size of 4 and the following stride values:
 * Stride 1: `ATCG TCGC CGCG GCGT CGTA GTAC TACG ACGA CGAT GATC ATCC TCCG`
 * Stride 2: `ATCG CGCG CGTA TACG CGAT ATCC`
 * Stride 3: `ATCG GCGT TACG GATC`
 * Stride 4: `ATCG CGTA CGAT`
 
Notice how the stride parameter affects the number of tokens created per length of input sequence. The impact of the choice of k-mer and stride values is discussed more in Section 5.7: Language Model Training. For now, understand that k-mer and stride values are hyperparameters that must be decided before training begins, and that the choice of k-mer and stride has an effect on compute time and performance.

### 1.2 Genomic Numericalization

Once we have decided on the k-mer and stride values to use in our tokenization, numericalizing is easy. We simply create a dictionary mapping each unique k-mer to an integer value. This creates the __vocabulary__ of the model - the total set of possible tokens. For a given k-mer length, the vocabulary will be $4^k + 1$ tokens. The $+ 1$ comes from adding a padding token, which will be important for batching sequences of different length. So for tokenization and numericalization of a sequence with k-mer length 3 and stride 2, we might see the following:

Sequence: `ATCGCGTACGATCCG`

Tokenization: `ATC CGC CGT TAC CGA ATC CCG`

Numericalization: `[5, 12, 8, 32, 27, 5, 14]`

Where the final numericalized list is the input to the model. Once a numericalized input is sent to the model, it is turned into a vector representation via an embedding. The integer values of the numericalized input correspond to rows in the embedding matrix. This is discussed further in Section 3.2: Embeddings.

### 1.3 Practical Tokenization

In practice I find k-mer lengths from 3-5 and stride values of 1-2 work best.

## 2. ULMFiT Overview

Now that we can process genomic data into a form we can feed to a model, we need to determine our strategy for training the model. Lets start by defining our end goal: We want to train a sequence model to classify genomic sequences using sequence input alone. This poses a potential problem. Sequence models tend to require a large amount of data to train effectively, and labeled genomic classification datasets can be small. The ULMFiT approach provides a solution to this. ULMFiT breaks training into three stages:

1. First we train a general domain language model using unsupervised on a large unlabeled corpus
2. We fine tune the general language model on the classification corpus to create a task specific language model
3. We fine tune the task specific language model for classification

Before going further, lets define the two types of models we will deal with. A __Language Model__ is a model that takes in a sequence of k-mer tokens and predicts the next token in the sequence. A __Classification Model__ is a model that takes in a sequence of tokens and predicts what category or class that sequence belongs to.

A language model is trained in an unsupervised fashion, meaning that no labeled data is required. Since the goal of the language model is to predict the next k-mer in a sequence, each k-mer becomes a correct output prediction for the sequence that preceeds it. This means we can generate huge amounts of paired data (input sequence + next k-mer) from any unlabeled genomic sequence.

A classification model is trained in a supervised fashion, requiring paired labeled data. For example if the task is promoter classification, all sequences in the classification dataset must be labeled as 0 or 1 for not-promoter or promoter.

The arthitectures for the Classification Model and the Language Model follow similar structures - the consist of an __Embedding__, an __Encoder__, and a __Linear Head__. On a high level, these layers function in the following ways:

 * Embedding: Converts the numericalized tokens of the input sequence into vector representations
 * Encoder: Processes the vectorized sequence into a hidden state
 * Linear Head: Uses the hidden state to make a classification decision.
 
When we move between stages, we transfer the learned weights from one model to the next. When we train the language models, we transfer all three sections of the model. When we transfer to the classification model, we only transfer the Embedding and the Encoder, as the classifcation model required a different linear head. Visually:

![](media/ulmfit1.png)

(Black arrows show transfer learning)
 
Model architecture is discussed in detail in Section 3.

### 2.1 General-Domain Language Model Training

In the first stage of training, we want to train a general genomic language model on a large unlabeled corpus. They key detail here is the training corpus used in this step can be any corpus in a similar domain to the classification corpus. So if for example we wanted to classify human genome sequences as either promoter or not-promoter, we could use the entire human genome to train the general domain language model. We could even go further and use an ensemble of genomes from animals phylogenically similar to humans. This allows us to generate large amounts of training data very easily.

The general domain language model will form the basis for all subsequent models. For this reason, we want the general domain model to be well trained. But what exactly does this mean? We want a model that understands the structure of genomic data and is able to pull meaning from nucleotide sequences. We use the language modeling task (predicting the next k-mer) as a proxy for this. In practice, I have seen that improving the general domain language model has a direct impact on improving performance of the classification model downstream. For this reason, it is worth investing in training a high performing general domain language model. Consequently, this step is actually the most time consuming step of the process. I typically invest 12+ hours into training the general domain language model, compared to 1-4 hours for fine tuning the task specific language model and 0.25-1 hours for training the classification model.

While training the general domain language model is computationally intensive, it only needs to be done once. If you train a human genome language model, that language model can be fine tuned for any number of downsteam tasks. This means the general domain language model has a high return on investment of compute time.

### 2.2 Task Specific Language Model

Once we have the general model trained, we want to fine tune it to the classification corpus to create a task specific language model. This is because no matter how general the general domain language model is, the classification dataset likely comes from a different distribution [1]. If we have a classification dataset specifically curated for a set of recognizable genomic classes, there are likely motifs and other structures in the sequence data that are more significant in the context of the classification dataset than in the general domain corpus. 

The task specific language model is initialized with the weights of the general domain language model. The full model (Embedding + Encoder + Linear Head) is transferred.

### 2.3 Task Specific Classification Model

Once we have trained the task specific language model, we can attempt our original goal: training a classification model. This model is initialized using the Embedding and the Encoder of the task specific language model. The Linear Head is not transferred, as the final classification task has changed. The Language Model produced a prediction vector corresponding to the length of the k-mer vocabulary (predicting the next k-mer), while the Classification Model produces a prediction vector with length equal to the number of classes in the classification dataset.

Initializing the classification model with the embedding and encoder of the task specific language model allows the classification model to train much faster while also being more robust to overfitting compared to training a model from scratch. Empirically I have found that models trained using transfer learning require much less regularization than models trained from scratch. This performance boost from pre-training and transfer learning is what allows ULMFiT to work so effectively on small datasets.

Transfer learning is an extremely important step for getting high quality results from training on small datasets. Transfer learning for deep learning genomics models has been done [3,7,11,12], but it is far from common. Many published methods train from scratch. I would expect pre-training to provide a general improvement to many methods.

## 3. Model Architecture

This section deals with the particulars of the language model and classification model architectures.

### 3.1 High Level Overview -  Language Models and Classification Models

As described in the previous section, ULMFiT uses a three step transfer learning process that utilizes two types of models: Language Models and Classification Models. These models are built using three sections: an __Embedding__, an __Encoder__, and a __Linear Head__. On a high level, these layers function in the following ways:

 * Embedding: Converts the tokens of the input sequence into vector representations
 * Encoder: Processes the vectorized sequence into a hidden state
 * Linear Head: Uses the hidden state to make a classification decision.

### 3.2 Embeddings

As described in Section 1, the ultimate input into the model is a vector of integers. For example:

`[5, 12, 8, 32, 27, 5, 14]`

Each integer value represents a k-mer in the vocabulary of our dataset. The sequence of integers represents a real sequence of k-mers. The first step of processing the input sequence is to convert the integer values into some sort of vector representation. A common way this is done in literature is to convert each token into a one hot encoded vector [2,3,4,5,6,7]. This works, but it is not a very rich representation. All the vectors are mutually orthagonal and don't contain any information about the relationships between k-mers.

By using an Embedding to represent k-mers as learned vectors, the model can learn more meaningful relationships between k-mers. This is also functionally identical to using one hot representations and passing them through a learned weight matrix.

The embedding weight matrix will have a size of `vocab x n_embedding` where `vocab` is the length of the model vocabulary and `n_embedding` is the length of the embedding vectors.

In practice the embedding is implemented using Pytorch's `nn.Embedding` module. An embedding vector length of 400 is typically used, but studies into the effect of embedding vector length have not been conducted.


### 3.3 LSTM Encoder

The encoder section is made of three stacked LSTM layers. This structure comes from the AWD-LSTM model [13] which is the standard model for ULMFiT [1]. An LSTM is used over a standard RNN as the update structure of the LSTM allows the model to retain information over longer sequences and filter information at each time step [14]. LSTMs are also less prone to vanishing gradients, a perpetual problem in standard RNNs. GRU units could likely be used in place of LSTMs, but this has not been tested.

The LSTM layers are structured such that the number of hidden units expand, then contract. A standard structure would be:
   1. LSTM(n_embedding, n_hidden)
   2. LSTM(n_hidden, n_hidden)
   3. LSTM(n_hidden, n_embedding)
   
Where `n_embedding` is the size of the embedding vectors, `n_hidden` is the number of hidden units in the LSTM stack, and `n_hidden > n_embedding`. The output of the final layer is set to a size of `n_embedding` to allow for weight tying when training the language models. See Section 3.4.1 for more information.

In practice an `n_hidden` value of 1150 is typically used. LSTMs are implemented using the standard Pytorch `nn.LSTM` module.


### 3.4 Linear Head

The linear head uses the hidden states output by the final LSTM layer to make predictions. Different linear heads are used for the Classification Model and the Language Model as each model is performing classification for different purposes, and use the hidden states from the final LSTM layer in different ways. The Language Model predicts the next k-mer in a genomic sequence, outputing a classification vector of length equal to the model vocabulary. The Language Model outputs a prediction for each time step in the input sequence, using each hidden state from the final LSTM layer to generate predictions.

The Classification Model makes classification predictions over the number of classes in the dataset. The classification model uses aggregates of all hidden states to produce a single prediction.

#### 3.4.1 Language Model Head

The Language Model linear head consists of just a single linear layer. The layer has a weight matrix of size `n_embedding x vocab`. The output is of length `vocab`, which makes sense as the model is predicting over k-mers in the vocab. The input size is `n_embedding`, which makes the size of the output weight matrix identical to the input embedding matrix, just with the dimensions flipped. This is intentional, as it allows us to tie the weights of the embedding to the softmax layer. This technique is motivated by [15,16] and found by [13] to lead to significantly improved language modeling performance. It also has the nice effect of reducing the number of parameters in the model.

The linear layer uses hidden states from all time steps to output predictions at every time step. So if a sequence section is 200 k-mers long, the model outputs a matrix of `200 x vocab` of next-k-mer predictions at every time step. This allows us to massively expand the amount of usable data we have for the language model. Each k-mer serves as the output value for the previous k-mer, and the input value for the subsequent k-mer. An important caveat to note is that this training approach is __not compatible__ with bidirectional LSTMs. If a bidirectional LSTM is used, the model will be predicting over k-mers it has already seen. A bidirectional model will achieve almost 100% accuracy while learning nothing.

In practice, the linear layer is implemented using Pytorch's `nn.Linear` module. The weights for the softmax layer are tied to the weights of the embedding layer. Typically bias is also included in the final layer, so strictly speaking there is not 100% weight tying between the embedding and the softmax layer.

#### 3.4.2 Classification Model Head

The linear head for the Classification Model is more complex than the linear head for the Language Model. This is due in part to the weight tying restriction on the complexity of the Language Model head - the Language Model must use a weight matrix the same size as the embedding, no more.

The Classification Model head consists of two linear layers with batchnorm layers in between. The classification head also uses the LSTM hidden states differently. Typically the final hidden state from the last LSTM layer is used for classification, but the most important parts of the sequence might be buried in the middle. Following the methods used in [1], we take the three vectors - the final hidden state, a maxpooling over all hidden states, and an average pooling over all hidden states - and concatenate them together. So for a vector containing hidden states at all time steps $ H = [h_{0}, ... h_{t}]$, we create $h_{c} = cat[h_{t}, maxpool(H), meanpool(H)]$ and use the vector $h_{c}$ as input to the linear head. $h_{c}$ will have a length of `n_embedding*3`. Since we transfer the learned wights from the embedding and encoder of the language model, we still use the LSTMs from the language model which output hidden states of length `n_embedding`.

The structure of the classification head is as follows:
1. Batchnorm1d(n_embedding*3)
2. Linear(n_embedding*3, n) + bias
3. ReLU
4. Batchnorm1d(n)
5. Linear(n, n_classes) + bias

In practice, the intermediate size `n` is typically 50, but can be tuned. Since the standard `n_embedding` size is 400, the input of length `n_embedding*3` will be of length 1200. The final output size `n_classes` is determined by the dataset.

### 3.5 Practical Model Parameters

To summarize the parameters used in practice

For the Embedding and Encoder layers:
   * Embeddings - size (vocab, 400)
   * LSTM 1 - size (400, 1150)
   * LSTM 2 - size (1150, 1150)
   * LSTM 3 - size (1150, 400)

For the Language Model Head:
   * Linear - size (400, vocab)
   
For the Classification Model Head:
   * Batchnorm1d 1 - size (1200)
   * Linear 1 - size (1200, 50)
   * Batchnorm1d 2 - size (50)
   * Linear 2 - size (50, n_classes)
   
Updating the diagram from earlier:

![](media/ulmfit2.png)

## 4. Regularization and Optimization

Effectively using regularization to improve model performance and prevent overfitting is one of the major results of [13]. Many different types of regularization are used in the AWD-LSTM model. This section covers the different forms of regularization and how they are implemented.

### 4.1 Dropout

Dropout, introduced by [17], is a standard form of regularization. Dropout uses a randomly selected mask to zero out activations, forcing the network to learn multiple paths from input to output. However a naive application of dropout functions poorly in recurrent models. Naive dropout - sampling a new dropout mask at every time step - disrupts the networks ability to retain long term dependencies [18]. To get around this, we need to be clever in how we apply dropout. Thankfully, all that hard work was done by [13] and now we can just follow their example. We apply dropout in four different ways:

#### Embedding Dropout

We apply dropout to the embedding matrix. Dropout is applied to the embedding matrix on the _k-mer level_. This means that every k-mer in the vocabulary has a probability of $p_{emb}$ of being completely zeroed out. The remaining word vectors are scaled by a factor of $\frac{1}{1-p_{emb}}$ to compensate. For a given batch of data, the same embedding dropout mask used for everything. This means that if a given k-mer vector is dropped out, that k-mer dissapears from the entire batch. This form of dropout is used by [13] and stems from [18].

#### Weight-Dropped LSTM

This form of dropout applies DropConnect [19] to the hidden-to-hidden weight matrices of the LSTM layers. There are two important points here. The first is that the dropout mask is sampled once at the start of the batch, then reused for all time steps within a batch. Using a constant dropout mask avoids the issues raised by [18] of dropout impacting the networks ability to retain long term dependencies. The second is that the dropout mask is applied to the _weights_ of the RNN, not the _activations_ moving through the RNN.

#### Variational Dropout

Variational Dropout, proposed by [18], is similar to DropConnect, except it is applied to the activations rather than the weights. Similar to DropConnect, a single dropout mask is sampled at the start of a batch and used for all inputs and outputs of the LSTM layers during the batch. Unlike DropConnect, which uses a single dropout mask for all elements in a batch, Variational Dropout uses a different mask for each element in a minibatch.

#### Standard Dropout

Regular old dropout is also used, but only in the linear head of the classification model. Dropout is used before each of the two linear layers in the classification head.


### 4.2 Weight Decay

Weight decay is used as another form of regularization. Weight decay penalizes the model for having large weights, and has been shown by [13] to improve the AWD-LSTM language model. An important point here is precisely _how_ weight decay is applied. There are two ways weight decay can be applied. There is the L2 Regularization method, where the sum of squared weights is added to directly to the loss: 

$L_{\text total} = \Big(L + \lambda||w||_2^2\Big)$

And the weight decay method, where the weight component is added during the gradient update.

$w_{i+1} = w_i - 2 \lambda w_i - \Big<\frac{\delta L}{\delta w}|_{w_i}\Big>$

From now on I will refer to the first method as L2 Regularization, and the second method as Weight Decay.

When using a simple optimizer like SGD, these methods are equivalent. However, we will be training the model using the Adam optimizer (see Section 5.1), which accumulates gradients. This means that L2 regularization will have a very different effect compared to Weight Decay. If we use L2 Regularization, the sum of squared weights becomes part of our loss value, and therefore part of our gradients. When the Adam optimizer accumulates gradients, the L2 Regularization term will be accumulated as well.

The Weight Decay method on the other hand is only added during the update step, so weight terms are not accumulated during the gradient. This difference was pointed out in [20], which showed the Weight Decay method (which they call AdamW) gave improved performance.

Weight Decay in Genomic-ULMFiT is implemented using the AdamW/Weight Decay method.

### 4.3 Activation Regularization

[13] implements another interesting form of regularization by adding an L2 penalty to the loss based on the activations of the model. This is implemented in two ways - Activation Regularization (AR) and Temporal Activation Regularization (TAR).

#### Activation Regularization

AR penalizes the model for having activations significantly larger than 0. AR is defined as $\alpha L_{2}(m \odot h_{t})$, where $m$ is the dropout mask, $h_{t}$ is the output activation at time $t$, $L_{2}(\cdot)$ is the sum of squared operation and $\alpha$ is the AR coefficient.

#### Temporal Activation Regularization

TAR [21] imposes a slowness constraint on the model by penalizing the difference in activation values between two time steps. TAR is defined as $\beta L_{2}(h_{t} - h_{t-1})$ where $h_{t}$ is the output activation at time $t$, $h_{t-1}$ is the output activation at time $t-1$ and $\beta$ is the TAR coefficient.

The L2 penalty for AR and TAR is used in the same way as L2 Regularization was defined in the previous section. Meaning the values for AR and TAR are added directly to the loss value before backpropagation. AR and TAR are embedded in the accumulated gradients.

### 4.4 Practical Regularization Parameters

We just covered many different types of regularization. This section summarizes practical values that I have found to work on genomic datasets. This comes with the caveat that I have not run serious ablation studies on all the regularization parameters. Many of these parameters come from the default ULMFiT values in [1].

#### Dropout Parameters

There are four dropout hyperparameters to set:
* $p_{emb}$ - dropout on the embedding
* $p_{weight}$ - dropout on the weights of the LSTM layers
* $p_{hidden}$ - variational dropout on the activations of the LSTM layers
* $p_{output}$ - standard dropout applied to activations in the linear head

It is important to consider both the magnitude of each dropout parameter and the relative ratios between dropout parameters. In practice I find it best to set values for all dropout parameters and tune a `drop_mult` parameter that scales all four dropout parameters while keeping the same ratio. I typically use the same ratios for all datasets, and tune `drop_mult` on a per dataset basis. With the caveat that I have not run serious ablations on dropout configurations, here are the parameters I typically use.

For genomic language models:
* $p_{emb} = 0.02$ 
* $p_{weight} = 0.15$
* $p_{hidden} = 0.1$
* $p_{output} = 0.25$
* $drop_mult = 0.15-0.35$

For genomic classification models:
* $p_{emb} = 0.1$ 
* $p_{weight} = 0.5$
* $p_{hidden} = 0.2$
* $p_{output} = 0.4$
* $drop_mult = 0.2 - 0.7$

#### Weight Decay

I typically use a weight decay coefficient of $wd = 1e-2$. If the model is struggling to train and changing dropout does not appear to help, I will lower weight decay to $1e-3$

#### AR and TAR

Following [13], the coefficients for AR and TAR are 2 and 1 respectively 